# Taller de la Ley de Zipf  

### Integrantes del grupo:
- Nombre 1  
- Nombre 2  
- Cristian Cardozo - cristiancardozo@javeriana.edu.co

📅 **Fecha de entrega:** [11/08/2025]


## Introducción

La Ley de Zipf establece que en una lengua natural, la frecuencia de las palabras es inversamente proporcional a su rango en una lista ordenada por frecuencia. Es decir, la segunda palabra más frecuente aparece aproximadamente la mitad de veces que la más frecuente, la tercera palabra aparece un tercio de veces, y así sucesivamente.

En este taller se aplicará esta ley al análisis de letras de canciones de distintos géneros musicales, combinando técnicas de análisis léxico, fonológico y textométrico. Se emplearán librerías de Python como `nltk`, `fonemas`, `textstat`, `matplotlib` y `pandas` para realizar estos análisis.

El propósito es comprender cómo se estructura el lenguaje en las canciones y qué patrones lingüísticos pueden encontrarse a través de estas herramientas computacionales.


## 1. Selección de canciones y carga de letras

Se seleccionop 1 cancion por año en colombia, ya sea la mas popular o la mas relevante. Cabe la pena aclarar que en caso de no encontrar la letra o no tener informacion desiciba para desicidir que cancion tomar, se procede a omitir ese año.

Esta informacion la consolidamos en un excel, con el año, el nombre de la cancion, sus interpretes y el genero, adenmas de la letra.

- 1945 - Porro  -  Se va el caimán - José María Peñaranda
- 1946 - Bambuco -  Soy Colombiano - Rafael Godoy (Garzón y Collazos)
- 1947 - Rumba criolla - La loca Margarita - Hermanas Garavito
- 1948 - Cumbia - Navidad Negra - Los Trovadores de Barú
- 1949 - Foxtrot tropical -  Pachito E’ché - Alex Tovar (Lucho Bermúdez y su Orq.)
- 1950 - Joropo -  Ay si, si - Luis Ariel Rey
- 1951 - Bambuco -  Los cisnes - Garzón y Collazos
- 1952 - Vallenato  -  Alicia la campesina - Los Vallenatos del Magdalena
- 1953 - Porro  -  El año viejo - Crescencio Salcedo
- 1954 - Porro -  La múcura - Lucho Bermúdez y Matilde Díaz
- 1955 - Merecumbé  -  Cosita linda - Pacho Galán
- 1956 - Vallenato -  Lirio rojo - Calixto Ochoa
- 1957 - Rumba criolla  -  Por vivir en Bogotá - Hermanas Garavito
- 1959 - Cumbia -  El pescador - José Barros (Los Trovadores de Barú)
- 1960 - Cumbia -  La piragua - José Barros (Los Black Stars)
- 1961 - Cumbia -  La pollera colorá - Wilson Choperena (Orq. de P. Salcedo)
- 1962 - Porro -  La paloma guarumera - Los Corraleros de Majagual
- 1963 - Vallenato -  La casa en el aire - Rafael Escalona (Bovea y sus Vallenatos)
- 1964 - Currulao  -  A la mina no voy - Leonor González Mina
- 1965 - Bambuco -  Pueblito viejo - Garzón y Collazos
- 1966 - Nueva ola  -  Llorando estoy - Vicky (Esperanza Acevedo)
- 1967 - Cumbia -  La piragua - Gabriel Romero & Los Black Stars
- 1968 - Vallenato  -  Pedazo de acordeón - Alejo Durán
- 1969 - Currulao  -  Mi Buenaventura - Peregoyo y su Combo Vacaná
- 1970 - Balada romántica -  Llévame contigo - Claudia de Colombia
- 1971 - Cumbia -  Los sabanales - Los Corraleros de Majagual (Calixto Ochoa)
- 1972 - Balada pop -  Alguien cantó una canción - Billy Pontoni
- 1973 - Salsa -  La sirena - Nelson y Sus Estrellas
- 1974 - Salsa -  Las caleñas son como las flores - The Latin Brothers (Joe Arroyo)
- 1975 - Salsa -  El preso - Fruko y Sus Tesos
- 1976 - Vallenato -  La creciente - Binomio de Oro de América
- 1977 - Balada  -  Llamarada - Isadora (Fernanda Bustos)
- 1978 - Bolero  -  María de los guardias - Helenita Vargas
- 1979 - Cumbia tropical -  Cariñito - Rodolfo Aicardi
- 1980 - Carranga  -  La cucharita - Jorge Velosa y Los Carrangueros
- 1981 - Joropo -  Ay mi llanura - Arnulfo Briceño
- 1982 - Vallenato -  Todo es para ti - Diomedes Díaz
- 1983 - Cumbia -  Golpe con golpe - Pastor López
- 1984 - Salsa -  Cali pachanguero - Grupo Niche
- 1985 - Balada pop -  ¿Dónde estará mi primavera? - Raul Santi
- 1986 - Salsa -  Rebelión - Joe Arroyo y La Verdad
- 1987 - Balada pop -  Como un picaflor - Raúl Santi
- 1988 - Rock en español -  La calle - Compañía Ilimitada
- 1989 - Balada  - La causa nacional - Sociedad Anónima
- 1990 - Salsa -  Una aventura - Grupo Niche
- 1991 - Cumbia - Colombia tierra querida - Lucho Bermúdez & Matilde Díaz
- 1993 - Vallenato fusión -  La gota fría - Carlos Vives
- 1994 - Vallenato -  El santo cachón - Los Embajadores Vallenatos
- 1995 - Pop latino  -  Estoy aquí - Shakira
- 1996 - Jazz-folk  - Los caminos de la vida - Los Diablitos (Omar Geles)
- 1997 - Tropipop  -  La tierra del olvido - Carlos Vives
- 1998 - Rock - Florecita rockera - Aterciopelados
- 1999 - Pop-rock  -  Me voy - Andrés Cepeda
- 2000 - Rock  -  Fíjate bien - Juanes
- 2001 - Pop rock  -  Suerte (Whenever, Wherever) - Shakira
- 2002 - Rock -  A Dios le pido - Juanes
- 2004 - Salsa  -  La pantera mambo - La 33
- 2005 - Pop rock  -  La camisa negra - Juanes
- 2007 - Hip-hop  -  Somos pacífico - ChocQuibTown
- 2008 - Electro-cumbia  -  Fuego - Bomba Estéreo
- 2009 - Electro-folk  -  Mi Kolombia - Systema Solar
- 2010 - Pop  -  Waka Waka - Shakira (ft. Freshlyground)
- 2011 - Hip-hop  -  Chocolate - Profetas
- 2012 - Swing  -  Suín Romanticón - Monsieur Periné
- 2013 - Pop-vallenato -  Volví a nacer - Carlos Vives
- 2014 - Reguetón -  6 AM - J Balvin (ft. Farruko)
- 2015 - Reguetón -  Ginza - J Balvin
- 2016 - Vallenato-pop -  La bicicleta - Carlos Vives & Shakira
- 2017 - Reguetón  -  Mi gente - J Balvin & Willy William
- 2018 - Reguetón -  Mi cama - Karol G
- 2019 - Reguetón-pop -  Tusa - Karol G (ft. Nicki Minaj)
- 2020 - Reguetón-pop -  Hawái - Maluma
- 2021 - Reguetón  -  Bichota - Karol G
- 2022 - Reguetón  -  Provenza - Karol G
- 2023 - Reguetón-pop  -  TQG (Te Quedó Grande) - Karol G & Shakira
- 2024 - Reguetón-pop  - LUNA - ATL Jacob & Feid
- 2025 - Pop - Enamorarte Mil Veces - Fonseca & Manuel Medrano

# ## 2. Análisis léxico y validación de la Ley de Zipf

Esta sección realiza la tokenización del texto de cada canción, el cálculo de la frecuencia de aparición de cada palabra y la visualización de la distribución de frecuencias.

Se utilizará `nltk` para el procesamiento de texto y `matplotlib` para graficar los resultados. El objetivo es verificar si las canciones siguen el comportamiento propuesto por la Ley de Zipf.

Se observará si la frecuencia de una palabra está inversamente relacionada con su posición en el ranking de palabras más usadas.